In [309]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [310]:
data = pd.read_csv('spam_or_not_spam.csv')

In [311]:
data.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [312]:
data=data.dropna()

In [313]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score

In [325]:
class Model:
    def __init__(self, model, vectorizer, data, random_seed=42):
        model_initializer ={
            "Tree": DecisionTreeClassifier(),
            'NB': MultinomialNB(),
            'LogReg': LogisticRegression()
        }
        vec_initializer ={
            "CountVectorizer": CountVectorizer(),
            "TF_IDF": TfidfVectorizer()
        }
        self.model = model_initializer.get(model)
        self.vectorizer = vec_initializer.get(vectorizer)
        self.data = data
        grid_tree = {'model__max_features': ['sqrt', 'log2'],
              'model__ccp_alpha': [0.1, .01, .001],
              'model__max_depth' : [100,300,500,None],
              'model__criterion' :['gini', 'entropy']
             }
        grid_logistic = {
            "model__penalty":['l1', 'l2'],
            'model__C':[0.1, 0.5, 1],
            'model__fit_intercept':[True, False]
        }
        grid_bayes = {
            "model__alpha":[0.01, 0.1, 1, 10]
        }
        grid_initializer = {
            "Tree": grid_tree,
            "NB": grid_bayes,
            "LogReg": grid_logistic
        }
        self.parameters = grid_initializer.get(model)
        
        self.pipe = Pipeline(
            steps = [('vectorize', self.vectorizer),
                     ('model', self.model)

            ]
        )

    def vectorize(self):
        X=self.data.email
        y=self.data.label
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
#         self.X_train = self.vectorizer.fit_transform(self.X_train)
#         self.X_test = self.vectorizer.transform(self.X_test)
        
#     def find_best_model(self):
#         gs = GridSearchCV(estimator=self.model, param_grid=self.parameters, cv=5, verbose=3)
#         gs.fit(self.X_train, self.y_train)
#         self.best_params=gs.best_params_
#         self.best_model = gs.best_estimator_
#         print(gs.best_score_)
#         print(self.best_params)
#         print(gs.best_estimator_)
    
    def pipeline(self):
        self.grid_pipeline = GridSearchCV(self.pipe, self.parameters,cv=5, verbose = 0)
        self.grid_pipeline.fit(self.X_train, self.y_train)
        
    def predict(self):
        
        self.y_pred = self.grid_pipeline.best_estimator_.predict(self.X_test)
        
    def evaluate(self):
#         print("F1-score: ", f1_score(self.y_test, self.y_pred))
#         cf_matrix = confusion_matrix(y_test,y_pred)
#         plt.figure(figsize=(4,3))
#         plt.title('Confusion Matrix Visualization')
#         sns.heatmap(cf_matrix, annot=True, fmt='', cmap='Blues')
        return f1_score(self.y_test, self.y_pred)

In [326]:
def run_all_models():
    models = ["NB", "LogReg", "Tree"]
    vectorizers = ["CountVectorizer", "TF_IDF"]
    results=[]
    for m in models:
        for v in vectorizers:
            model = Model(m,v,data)
            model.vectorize()
            model.pipeline()
            model.predict()
            f_score = model.evaluate()
            results.append((model.grid_pipeline.best_estimator_[1], model.vectorizer, f_score))
            
    return results

In [327]:
results = run_all_models()

In [328]:
results

[(MultinomialNB(alpha=0.1), CountVectorizer(), 0.9847715736040609),
 (MultinomialNB(alpha=0.01), TfidfVectorizer(), 0.9743589743589743),
 (LogisticRegression(C=0.5), CountVectorizer(), 0.9847715736040609),
 (LogisticRegression(C=1, fit_intercept=False),
  TfidfVectorizer(),
  0.9071038251366119),
 (DecisionTreeClassifier(ccp_alpha=0.001, criterion='entropy',
                         max_features='sqrt'),
  CountVectorizer(),
  0.7916666666666667),
 (DecisionTreeClassifier(ccp_alpha=0.001, criterion='entropy', max_depth=100,
                         max_features='sqrt'),
  TfidfVectorizer(),
  0.7643979057591622)]